# Introduction
in notebook '42. Lipid metabolism-I' we tackled the production of the first few fatty acids that are needed. To prevent that notebook from becoming too large, here I will include the biosynthesis of the last fatty acid chains that our model needs to make. 

Here I will ensure the Iso FAs can be made, both even and uneven. 


In [1]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction, Metabolite

In [4]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [5]:
model_e_coli = cameo.load_model('iML1515')

In [6]:
model_b_sub = cameo.load_model('iYO844')

### Iso-Branched, even chained fatty acids
In this category, we need to ensure the production of iso-C15:0 and isoC17:0 fatty acids (i.e. the main chain is even length). In general, the even chain iso fatty acids are made from leucine that is converted into a primer molecule that can then go through the cycles of elongation until the final chain length is achieved. 

The leucine is converted into 3-Methylbutanoyl-CoA (aka Isovaleryl-CoA, Kegg ID C02939). Checking in the model, we already have this metaboltite: ivcoa_c. I will check it is properly converted from leucine first. Leucine should be converted into 4mop_c and then into the ivcoa_c metabolite. The conversion of 4mop_c into ivcoa_c is correctly present, the conversion of leucine into 4mop_c is also there. So now I need to add a reaction that converts ivcoa_c into ivACP, which can then enter the cycles of elongation.


In [7]:
#add ivACP_c meta,bolite
model.add_metabolites(Metabolite(id='ivACP_c', name = '3-Methylbutanoyl-ACP', compartment= 'c', formula = 'C16H29N2O8PRS', charge = -1))

In [8]:
model.metabolites.ivACP_c.annotation['sbo'] = 'SBO:0000176'

In [9]:
#then add conversion of ivcoa into ivACP

In [10]:
model.add_reaction(Reaction(id='IVCOATA', name = '3-methylbutyryl-CoA:[acyl-carrier-protein] transferase'))

In [11]:
model.reactions.IVCOATA.annotation['sbo'] = 'SBO:0000176'

In [12]:
model.reactions.IVCOATA.add_metabolites({
    model.metabolites.ACP_c:-1,
    model.metabolites.ivcoa_c:-1,
    model.metabolites.coa_c:1,
    model.metabolites.ivACP_c:1,
    model.metabolites.h_c:1
})

In [13]:
model.reactions.IVCOATA.bounds = (-1000,1000)

In [14]:
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now that the primer molecule is ready, we can start the elongation process with malonyl-ACP to iso-C6:0.

In [15]:
#add condenstation of s3bmcoa-ACP with malonyl ACP to 3-oxo3mnoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o3mhACP_c', name = '3-oxo3methylhexanoyl-ACP', compartment = 'c', charge =-1, formula = 'C18H31N2O9PRS'))

In [16]:
model.metabolites.get_by_id('3o3mhACP_c').annotation['sbo'] = 'SBO:0000176'

In [17]:
#add reaction
model.add_reaction(Reaction(id='3O3MAS70', name = '3-oxo3methylhexanoyl-[acyl-carrier-protein] synthase (iso-C7:0)'))

In [18]:
model.reactions.get_by_id('3O3MAS70').annotation['sbo'] = 'SBO:0000176'

In [19]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAS70').gene_reaction_rule = 'RTMO12345'

In [20]:
model.reactions.get_by_id('3O3MAS70').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.ivACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o3mhACP_c'):1,
    model.metabolites.h_c:-1
})

In [21]:
#next add reaction with NADPH
#first add 3-Hydroxy-3-methylheptanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h3mhACP_c', name = '3-Hydroxy-3-methylhexanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C18H33N2O9PRS'))

In [22]:
model.metabolites.get_by_id('3h3mhACP_c').annotation['sbo'] = 'SBO:0000247'

In [23]:
#add reaction
model.add_reaction(Reaction(id='3O3MAR70', name = ' 3-oxo-3-methyl-acyl-[acyl-carrier-protein] reductase (iso-C7:0)'))

In [24]:
model.reactions.get_by_id('3O3MAR70').annotation['sbo'] = 'SBO:0000176'

In [25]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAR70').gene_reaction_rule = 'RTMO12345'

In [26]:
model.reactions.get_by_id('3O3MAR70').add_metabolites({
    model.metabolites.get_by_id('3o3mhACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h3mhACP_c'):1,
    model.metabolites.h_c:-1
})

In [27]:
#next add the t3mh2eACP_c metabolite
model.add_metabolites(Metabolite(id='t3mh2eACP_c', name = 'Trans-3-methylhex-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C18H31N2O8PRS'))

In [28]:
model.metabolites.get_by_id('t3mh2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [29]:
#add reaction
model.add_reaction(Reaction(id='3H3MAD70', name = '3-hydroxy-3-methylacyl-[acyl-carrier-protein] dehydratase (iso-C7:0)'))

In [30]:
model.reactions.get_by_id('3H3MAD70').annotation['sbo'] = 'SBO:0000176'

In [31]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3H3MAD70').gene_reaction_rule = 'RTMO12345'

In [32]:
model.reactions.get_by_id('3H3MAD70').add_metabolites({
    model.metabolites.get_by_id('3h3mhACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t3mh2eACP_c:1
})

In [33]:
#last step: convert to 3-methylhexanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 3-methylhexanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='3mhexACP_c', name = '3-methyl-hexanoyl-ACP(iso-C7:0ACP)', compartment = 'c', charge = -1, formula = 'C18H33N2O8PRS' ))

In [34]:
model.metabolites.get_by_id('3mhexACP_c').annotation['sbo'] = 'SBO:0000247'

In [35]:
#add reaction with NADH: E3MAR70x
model.add_reaction(Reaction(id='E3MAR70x', name = '3-Methyl-Hexanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [36]:
model.reactions.E3MAR70x.annotation['sbo'] = 'SBO:0000176'

In [37]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR70x.gene_reaction_rule = 'RTMO12345'

In [38]:
model.reactions.E3MAR70x.add_metabolites({
    model.metabolites.t3mh2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('3mhexACP_c'):1,
    model.metabolites.h_c:-1
})

In [39]:
#add reaction with NADH: E3MAR70y
model.add_reaction(Reaction(id='E3MAR70y', name = '3-Methyl-Hexanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [40]:
model.reactions.E3MAR70y.annotation['sbo'] = 'SBO:0000176'

In [41]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR70y.gene_reaction_rule = 'RTMO12345'

In [42]:
model.reactions.E3MAR70y.add_metabolites({
    model.metabolites.t3mh2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3mhexACP_c'):1,
    model.metabolites.h_c:-1
})

In [43]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished to iso-C7:0. Now onto iso-C9:0. 

In [44]:
#add condenstation of 3mhexACP with malonyl ACP to 3-oxo-3methyl-octanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o3moACP_c', name = '3-oxo3methyloctanoyl-ACP', compartment = 'c', charge =-1, formula = 'C20H35N2O9PRS'))

In [45]:
model.metabolites.get_by_id('3o3moACP_c').annotation['sbo'] = 'SBO:0000176'

In [46]:
#add reaction
model.add_reaction(Reaction(id='3O3MAS90', name = '3-oxo3methyloctanoyl-[acyl-carrier-protein] synthase (iso-C9:0)'))

In [47]:
model.reactions.get_by_id('3O3MAS90').annotation['sbo'] = 'SBO:0000176'

In [48]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAS90').gene_reaction_rule = 'RTMO12345'

In [49]:
model.reactions.get_by_id('3O3MAS90').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('3mhexACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o3moACP_c'):1,
    model.metabolites.h_c:-1
})

In [50]:
#next add reaction with NADPH
#first add 3-Hydroxy-3-methyloctanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h3moACP_c', name = '3-Hydroxy-3-methyloctanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C20H37N2O9PRS'))

In [51]:
model.metabolites.get_by_id('3h3moACP_c').annotation['sbo'] = 'SBO:0000247'

In [52]:
#add reaction
model.add_reaction(Reaction(id='3O3MAR90', name = ' 3-oxo-3-methyl-acyl-[acyl-carrier-protein] reductase (iso-C9:0)'))

In [53]:
model.reactions.get_by_id('3O3MAR90').annotation['sbo'] = 'SBO:0000176'

In [54]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAR90').gene_reaction_rule = 'RTMO12345'

In [55]:
model.reactions.get_by_id('3O3MAR90').add_metabolites({
    model.metabolites.get_by_id('3o3moACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h3moACP_c'):1,
    model.metabolites.h_c:-1
})

In [56]:
#next add the t3mh2eACP_c metabolite
model.add_metabolites(Metabolite(id='t3mo2eACP_c', name = 'Trans-3-methyloct-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C20H35N2O8PRS'))

In [57]:
model.metabolites.get_by_id('t3mo2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [58]:
#add reaction
model.add_reaction(Reaction(id='3H3MAD90', name = '3-hydroxy-3-methylacyl-[acyl-carrier-protein] dehydratase (iso-C9:0)'))

In [59]:
model.reactions.get_by_id('3H3MAD90').annotation['sbo'] = 'SBO:0000176'

In [60]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3H3MAD90').gene_reaction_rule = 'RTMO12345'

In [61]:
model.reactions.get_by_id('3H3MAD90').add_metabolites({
    model.metabolites.get_by_id('3h3moACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t3mo2eACP_c:1
})

In [62]:
#last step: convert to 3-methyloctanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 3-methyloctanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='3moctACP_c', name = '3-methyl-octanoyl-ACP(iso-C9:0ACP)', compartment = 'c', charge = -1, formula = 'C20H37N2O8PRS' ))

In [63]:
model.metabolites.get_by_id('3moctACP_c').annotation['sbo'] = 'SBO:0000247'

In [64]:
#add reaction with NADH: EMAR90x
model.add_reaction(Reaction(id='E3MAR90x', name = '3-Methyl-Octanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [65]:
model.reactions.E3MAR90x.annotation['sbo'] = 'SBO:0000176'

In [66]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR90x.gene_reaction_rule = 'RTMO12345'

In [67]:
model.reactions.E3MAR90x.add_metabolites({
    model.metabolites.t3mo2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('3moctACP_c'):1,
    model.metabolites.h_c:-1
})

In [68]:
#add reaction with NADH: E3MAR90y
model.add_reaction(Reaction(id='E3MAR90y', name = '3-Methyl-Octanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [69]:
model.reactions.E3MAR90y.annotation['sbo'] = 'SBO:0000176'

In [70]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR90y.gene_reaction_rule = 'RTMO12345'

In [71]:
model.reactions.E3MAR90y.add_metabolites({
    model.metabolites.t3mo2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3moctACP_c'):1,
    model.metabolites.h_c:-1
})

In [72]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished to iso-C9:0, now onto iso-C11:0.

In [73]:
#add condenstation of 3moctACP with malonyl ACP to 3-oxo-3-methyl-decanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o3mdACP_c', name = '3-oxo3methyldecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C22H39N2O9PRS'))

In [74]:
model.metabolites.get_by_id('3o3mdACP_c').annotation['sbo'] = 'SBO:0000176'

In [75]:
#add reaction
model.add_reaction(Reaction(id='3O3MAS110', name = '3-oxo3methyldecanoyl-[acyl-carrier-protein] synthase (iso-C11:0)'))

In [76]:
model.reactions.get_by_id('3O3MAS110').annotation['sbo'] = 'SBO:0000176'

In [77]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAS110').gene_reaction_rule = 'RTMO12345'

In [78]:
model.reactions.get_by_id('3O3MAS110').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('3moctACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o3mdACP_c'):1,
    model.metabolites.h_c:-1
})

In [79]:
#next add reaction with NADPH
#first add 3-Hydroxy-3-methyldecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h3mdACP_c', name = '3-Hydroxy-3-methyldecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C22H41N2O9PRS'))

In [80]:
model.metabolites.get_by_id('3h3mdACP_c').annotation['sbo'] = 'SBO:0000247'

In [81]:
#add reaction
model.add_reaction(Reaction(id='3O3MAR110', name = ' 3-oxo-3-methyl-acyl-[acyl-carrier-protein] reductase (iso-C11:0)'))

In [82]:
model.reactions.get_by_id('3O3MAR110').annotation['sbo'] = 'SBO:0000176'

In [83]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAR110').gene_reaction_rule = 'RTMO12345'

In [84]:
model.reactions.get_by_id('3O3MAR110').add_metabolites({
    model.metabolites.get_by_id('3o3mdACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h3mdACP_c'):1,
    model.metabolites.h_c:-1
})

In [85]:
#next add the t3md2eACP_c metabolite
model.add_metabolites(Metabolite(id='t3md2eACP_c', name = 'Trans-3-methyldec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C22H39N2O8PRS'))

In [86]:
model.metabolites.get_by_id('t3md2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [87]:
#add reaction
model.add_reaction(Reaction(id='3H3MAD110', name = '3-hydroxy-3-methylacyl-[acyl-carrier-protein] dehydratase (iso-C11:0)'))

In [88]:
model.reactions.get_by_id('3H3MAD110').annotation['sbo'] = 'SBO:0000176'

In [89]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3H3MAD110').gene_reaction_rule = 'RTMO12345'

In [90]:
model.reactions.get_by_id('3H3MAD110').add_metabolites({
    model.metabolites.get_by_id('3h3mdACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t3md2eACP_c:1
})

In [91]:
#last step: convert to 3-methyldecanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 3-methyldecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='3mdecACP_c', name = '3-methyl-decanoyl-ACP(iso-C11:0ACP)', compartment = 'c', charge = -1, formula = 'C22H41N2O8PRS' ))

In [92]:
model.metabolites.get_by_id('3mdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [93]:
#add reaction with NADH: E3MAR90x
model.add_reaction(Reaction(id='E3MAR110x', name = '3-Methyl-Decanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [94]:
model.reactions.E3MAR110x.annotation['sbo'] = 'SBO:0000176'

In [95]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR110x.gene_reaction_rule = 'RTMO12345'

In [96]:
model.reactions.E3MAR110x.add_metabolites({
    model.metabolites.t3md2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('3mdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [97]:
model.reactions.E3MAR110x.check_mass_balance()

{}

In [98]:
#add reaction with NADH: E3MAR110y
model.add_reaction(Reaction(id='E3MAR110y', name = '3-Methyl-Decanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [99]:
model.reactions.E3MAR110y.annotation['sbo'] = 'SBO:0000176'

In [100]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR110y.gene_reaction_rule = 'RTMO12345'

In [101]:
model.reactions.E3MAR110y.add_metabolites({
    model.metabolites.t3md2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3mdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [102]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished to isoC11:0. Now onto isoC13:0.

In [103]:
#add condenstation of 3mdecACP with malonyl ACP to 3-oxo-3-methyl-dodecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o3mddACP_c', name = '3-oxo3methyldodecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C24H43N2O9PRS'))

In [104]:
model.metabolites.get_by_id('3o3mddACP_c').annotation['sbo'] = 'SBO:0000176'

In [105]:
#add reaction
model.add_reaction(Reaction(id='3O3MAS130', name = '3-oxo3methyldodecanoyl-[acyl-carrier-protein] synthase (iso-C13:0)'))

In [106]:
model.reactions.get_by_id('3O3MAS130').annotation['sbo'] = 'SBO:0000176'

In [107]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAS130').gene_reaction_rule = 'RTMO12345'

In [108]:
model.reactions.get_by_id('3O3MAS130').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('3mdecACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o3mddACP_c'):1,
    model.metabolites.h_c:-1
})

In [109]:
#next add reaction with NADPH
#first add 3-Hydroxy-3-methyldodecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h3mddACP_c', name = '3-Hydroxy-3-methyldodecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C24H45N2O9PRS'))

In [110]:
model.metabolites.get_by_id('3h3mddACP_c').annotation['sbo'] = 'SBO:0000247'

In [111]:
#add reaction
model.add_reaction(Reaction(id='3O3MAR130', name = ' 3-oxo-3-methyl-acyl-[acyl-carrier-protein] reductase (iso-C13:0)'))

In [112]:
model.reactions.get_by_id('3O3MAR130').annotation['sbo'] = 'SBO:0000176'

In [113]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAR130').gene_reaction_rule = 'RTMO12345'

In [114]:
model.reactions.get_by_id('3O3MAR130').add_metabolites({
    model.metabolites.get_by_id('3o3mddACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h3mddACP_c'):1,
    model.metabolites.h_c:-1
})

In [115]:
#next add the t3md2eACP_c metabolite
model.add_metabolites(Metabolite(id='t3mdd2eACP_c', name = 'Trans-3-methyldodec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C24H43N2O8PRS'))

In [116]:
model.metabolites.get_by_id('t3mdd2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [117]:
#add reaction
model.add_reaction(Reaction(id='3H3MAD130', name = '3-hydroxy-3-methylacyl-[acyl-carrier-protein] dehydratase (iso-C13:0)'))

In [118]:
model.reactions.get_by_id('3H3MAD130').annotation['sbo'] = 'SBO:0000176'

In [119]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3H3MAD130').gene_reaction_rule = 'RTMO12345'

In [120]:
model.reactions.get_by_id('3H3MAD130').add_metabolites({
    model.metabolites.get_by_id('3h3mddACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t3mdd2eACP_c:1
})

In [121]:
#last step: convert to 3-methyldodecanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 3-methyldodecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='3mddecACP_c', name = '3-methyl-dodecanoyl-ACP(iso-C13:0ACP)', compartment = 'c', charge = -1, formula = 'C24H45N2O8PRS' ))

In [122]:
model.metabolites.get_by_id('3mddecACP_c').annotation['sbo'] = 'SBO:0000247'

In [123]:
#add reaction with NADH: E3MAR1300x
model.add_reaction(Reaction(id='E3MAR130x', name = '3-Methyl-Dodecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [124]:
model.reactions.E3MAR130x.annotation['sbo'] = 'SBO:0000176'

In [125]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR130x.gene_reaction_rule = 'RTMO12345'

In [126]:
model.reactions.E3MAR130x.add_metabolites({
    model.metabolites.t3mdd2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('3mddecACP_c'):1,
    model.metabolites.h_c:-1
})

In [127]:
#add reaction with NADH: E3MAR130y
model.add_reaction(Reaction(id='E3MAR130y', name = '3-Methyl-Dodecanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [128]:
model.reactions.E3MAR130y.annotation['sbo'] = 'SBO:0000176'

In [129]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR130y.gene_reaction_rule = 'RTMO12345'

In [130]:
model.reactions.E3MAR130y.add_metabolites({
    model.metabolites.t3mdd2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3mddecACP_c'):1,
    model.metabolites.h_c:-1
})

In [131]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished up to iso-C13:0. Now onto isoC15:0.

In [132]:
#add condenstation of 3mddecACP with malonyl ACP to 3-oxo-3-methyl-tetradecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o3mtdACP_c', name = '3-oxo3methyltetradecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C26H47N2O9PRS'))

In [133]:
model.metabolites.get_by_id('3o3mtdACP_c').annotation['sbo'] = 'SBO:0000176'

In [134]:
#add reaction
model.add_reaction(Reaction(id='3O3MAS150', name = '3-oxo3methyltetradecanoyl-[acyl-carrier-protein] synthase (iso-C15:0)'))

In [135]:
model.reactions.get_by_id('3O3MAS150').annotation['sbo'] = 'SBO:0000176'

In [136]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAS150').gene_reaction_rule = 'RTMO12345'

In [137]:
model.reactions.get_by_id('3O3MAS150').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('3mddecACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o3mtdACP_c'):1,
    model.metabolites.h_c:-1
})

In [138]:
#next add reaction with NADPH
#first add 3-Hydroxy-3-methyltetradecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h3mtdACP_c', name = '3-Hydroxy-3-methyltetradecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C26H49N2O9PRS'))

In [139]:
model.metabolites.get_by_id('3h3mtdACP_c').annotation['sbo'] = 'SBO:0000247'

In [140]:
#add reaction
model.add_reaction(Reaction(id='3O3MAR150', name = ' 3-oxo-3-methyl-acyl-[acyl-carrier-protein] reductase (iso-C15:0)'))

In [141]:
model.reactions.get_by_id('3O3MAR150').annotation['sbo'] = 'SBO:0000176'

In [142]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAR150').gene_reaction_rule = 'RTMO12345'

In [143]:
model.reactions.get_by_id('3O3MAR150').add_metabolites({
    model.metabolites.get_by_id('3o3mtdACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h3mtdACP_c'):1,
    model.metabolites.h_c:-1
})

In [144]:
#next add the t3mtd2eACP_c metabolite
model.add_metabolites(Metabolite(id='t3mtd2eACP_c', name = 'Trans-3-methyltetradec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C26H47N2O8PRS'))

In [145]:
model.metabolites.get_by_id('t3mtd2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [146]:
#add reaction
model.add_reaction(Reaction(id='3H3MAD150', name = '3-hydroxy-3-methylacyl-[acyl-carrier-protein] dehydratase (iso-C15:0)'))

In [147]:
model.reactions.get_by_id('3H3MAD150').annotation['sbo'] = 'SBO:0000176'

In [148]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3H3MAD150').gene_reaction_rule = 'RTMO12345'

In [149]:
model.reactions.get_by_id('3H3MAD150').add_metabolites({
    model.metabolites.get_by_id('3h3mtdACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t3mtd2eACP_c:1
})

In [150]:
#last step: convert to 3-methyltetradecanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 3-methyltetradecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='3mtdecACP_c', name = '3-methyl-tetradecanoyl-ACP(iso-C15:0ACP)', compartment = 'c', charge = -1, formula = 'C26H49N2O8PRS' ))

In [151]:
model.metabolites.get_by_id('3mtdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [152]:
#add reaction with NADH: E3MAR1300x
model.add_reaction(Reaction(id='E3MAR150x', name = '3-Methyl-Tetradecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [153]:
model.reactions.E3MAR150x.annotation['sbo'] = 'SBO:0000176'

In [154]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR150x.gene_reaction_rule = 'RTMO12345'

In [155]:
model.reactions.E3MAR150x.add_metabolites({
    model.metabolites.t3mtd2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('3mtdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [156]:
#add reaction with NADH: E3MAR150y
model.add_reaction(Reaction(id='E3MAR150y', name = '3-Methyl-Tetradecanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [157]:
model.reactions.E3MAR150y.annotation['sbo'] = 'SBO:0000176'

In [158]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR150y.gene_reaction_rule = 'RTMO12345'

In [159]:
model.reactions.E3MAR150y.add_metabolites({
    model.metabolites.t3mtd2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3mtdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [160]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished to iso-C15:0. Now onto the last, iso-C17:0.

In [161]:
#add condenstation of 3mtdecACP with malonyl ACP to 3-oxo-3-methyl-hexadecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o3mhdACP_c', name = '3-oxo3methylhexadecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C28H51N2O9PRS'))

In [162]:
model.metabolites.get_by_id('3o3mhdACP_c').annotation['sbo'] = 'SBO:0000176'

In [163]:
#add reaction
model.add_reaction(Reaction(id='3O3MAS170', name = '3-oxo3methylhexadecanoyl-[acyl-carrier-protein] synthase (iso-C17:0)'))

In [164]:
model.reactions.get_by_id('3O3MAS170').annotation['sbo'] = 'SBO:0000176'

In [165]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAS170').gene_reaction_rule = 'RTMO12345'

In [166]:
model.reactions.get_by_id('3O3MAS170').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('3mtdecACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o3mhdACP_c'):1,
    model.metabolites.h_c:-1
})

In [167]:
#next add reaction with NADPH
#first add 3-Hydroxy-3-methylhexadecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h3mhdACP_c', name = '3-Hydroxy-3-methylhexadecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C28H53N2O9PRS'))

In [168]:
model.metabolites.get_by_id('3h3mhdACP_c').annotation['sbo'] = 'SBO:0000247'

In [169]:
#add reaction
model.add_reaction(Reaction(id='3O3MAR170', name = ' 3-oxo-3-methyl-acyl-[acyl-carrier-protein] reductase (iso-C17:0)'))

In [170]:
model.reactions.get_by_id('3O3MAR170').annotation['sbo'] = 'SBO:0000176'

In [171]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3O3MAR170').gene_reaction_rule = 'RTMO12345'

In [172]:
model.reactions.get_by_id('3O3MAR170').add_metabolites({
    model.metabolites.get_by_id('3o3mhdACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h3mhdACP_c'):1,
    model.metabolites.h_c:-1
})

In [173]:
#next add the t2mhd2eACP_c metabolite
model.add_metabolites(Metabolite(id='t3mhd2eACP_c', name = 'Trans-3-methylhexadec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C28H51N2O8PRS'))

In [174]:
model.metabolites.get_by_id('t3mhd2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [175]:
#add reaction
model.add_reaction(Reaction(id='3H3MAD170', name = '3-hydroxy-3-methylacyl-[acyl-carrier-protein] dehydratase (iso-C17:0)'))

In [176]:
model.reactions.get_by_id('3H3MAD170').annotation['sbo'] = 'SBO:0000176'

In [177]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3H3MAD170').gene_reaction_rule = 'RTMO12345'

In [178]:
model.reactions.get_by_id('3H3MAD170').add_metabolites({
    model.metabolites.get_by_id('3h3mhdACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t3mhd2eACP_c:1
})

In [179]:
#last step: convert to 3-methylhexadecanoyl-ACP 
#add both with NADH and NADPH.
#first add the 3-methylhexadecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='3mhdecACP_c', name = '3-methyl-hexadecanoyl-ACP(iso-C17:0ACP)', compartment = 'c', charge = -1, formula = 'C28H53N2O8PRS' ))

In [180]:
model.metabolites.get_by_id('3mhdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [181]:
#add reaction with NADH: E3MAR170x
model.add_reaction(Reaction(id='E3MAR170x', name = '3-Methyl-Hexadecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [182]:
model.reactions.E3MAR170x.annotation['sbo'] = 'SBO:0000176'

In [183]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR170x.gene_reaction_rule = 'RTMO12345'

In [184]:
model.reactions.E3MAR170x.add_metabolites({
    model.metabolites.t3mhd2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('3mhdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [185]:
#add reaction with NADH: E3MAR170y
model.add_reaction(Reaction(id='E3MAR170y', name = '3-Methyl-Hexadecanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [186]:
model.reactions.E3MAR170y.annotation['sbo'] = 'SBO:0000176'

In [187]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.E3MAR170y.gene_reaction_rule = 'RTMO12345'

In [188]:
model.reactions.E3MAR170y.add_metabolites({
    model.metabolites.t3mhd2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3mhdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [189]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished until iso-C17:0.
Just check the model prediction is still oke.

In [190]:
model.optimize()

,fluxes,reduced_costs
IDPh,1.770060,0.000000
CAT,0.019791,0.000000
PDHam1hi,0.000000,0.000000
HYDA,0.000000,-0.000000
MALHYDRO,0.000000,0.000000
...,...,...
3O3MAS170,0.000000,-0.057007
3O3MAR170,0.000000,-0.005701
3H3MAD170,0.000000,0.000000
E3MAR170x,0.000000,-0.013682


## Iso-Branched, odd chained fatty acids¶
In this category, we need to ensure the production of iso-C14:0 and iso-C16:0 fatty acids (i.e. the main chain is odd length). In general, the odd chain iso fatty acids are made from valine that is converted into a primer molecule that can then go through the cycles of elongation until the final chain length is achieved.

In the case here, the valine is converted into isobutryl-coa, which gets bound to an ACP and can start cycles of elongation. the ibcoa_c metabolite is already present. I will check it is properly made from valine, and than make its conversion into ibACP_c, before the elongation cycles start. Seems the conversion from Valine is possible and correct, and so I will just add the conversion of ibcoa_c into ibACP_c. And from there start the elongation cycles.

In [189]:
#add ibACP_c meta,bolite
model.add_metabolites(Metabolite(id='ibACP_c', name = '2-Methylpropanoyl-ACP', compartment= 'c', formula = 'C15H27N2O8PRS', charge = -1))

In [190]:
model.metabolites.ibACP_c.annotation['sbo'] = 'SBO:0000176'

In [191]:
#then add conversion of ibcoa into ibACP

In [192]:
model.add_reaction(Reaction(id='IBCOATA', name = 'isobutyryl-CoA:[acyl-carrier-protein] transferase'))

Ignoring reaction 'IBCOATA' since it already exists.


In [193]:
model.reactions.IBCOATA.annotation['sbo'] = 'SBO:0000176'

In [194]:
model.reactions.IBCOATA.add_metabolites({
    model.metabolites.ACP_c:-1,
    model.metabolites.ibcoa_c:-1,
    model.metabolites.coa_c:1,
    model.metabolites.ibACP_c:1,
    model.metabolites.h_c:1
})

In [195]:
model.reactions.IBCOATA.bounds = (-1000,1000)

In [196]:
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished the primer molecule, now it can condense with Malonyl-CoA to give iso-C6:0. 

In [197]:
#add condenstation of 2-methyl-prop-ACP with malonyl ACP to 3-oxo-2-methyl pentanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mpentACP_c', name = '3-oxo-2-methylpentanoyl-ACP', compartment = 'c', charge =-1, formula = 'C17H29O9N2PRS'))

In [198]:
model.metabolites.get_by_id('3o2mpentACP_c').annotation['sbo'] = 'SBO:0000176'

In [199]:
#add reaction
model.add_reaction(Reaction(id='3OMAS60', name = '3-oxo-2-Methylpentanoyl-[acyl-carrier-protein] synthase (iso-C6:0)'))

In [200]:
model.reactions.get_by_id('3OMAS60').annotation['sbo'] = 'SBO:0000176'

In [201]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS60').gene_reaction_rule = 'RTMO12345'

In [202]:
model.reactions.get_by_id('3OMAS60').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.ibACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mpentACP_c'):1,
    model.metabolites.h_c:-1
})

In [203]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methylpentanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mpentACP_c', name = '3-Hydroxy-2-methylpentanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C17H31O9N2PRS'))

In [204]:
model.metabolites.get_by_id('3h2mpentACP_c').annotation['sbo'] = 'SBO:0000247'

In [205]:
#add reaction
model.add_reaction(Reaction(id='3OMAR60', name = ' 3-oxo-2-methylacyl-[acyl-carrier-protein] reductase (iso-C6:0)'))

In [206]:
model.reactions.get_by_id('3OMAR60').annotation['sbo'] = 'SBO:0000176'

In [207]:
#assume GPR is the same FAsynthase: RTMO12345
model.reactions.get_by_id('3OMAR60').gene_reaction_rule = 'RTMO12345'

In [208]:
model.reactions.get_by_id('3OMAR60').add_metabolites({
    model.metabolites.get_by_id('3o2mpentACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mpentACP_c'):1,
    model.metabolites.h_c:-1
})

In [209]:
#next add the tpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mpent2eACP_c', name = 'Trans-2-methyl-Pent-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C17H29O8N2PRS'))

In [210]:
model.metabolites.get_by_id('t2mpent2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [211]:
#add reaction
model.add_reaction(Reaction(id='3HMAD60', name = '3-hydroxy-2-methyl-acyl-[acyl-carrier-protein] dehydratase (iso-C6:0)'))

In [212]:
model.reactions.get_by_id('3HMAD60').annotation['sbo'] = 'SBO:0000176'

In [213]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD60').gene_reaction_rule = 'RTMO12345'

In [214]:
model.reactions.get_by_id('3HMAD60').add_metabolites({
    model.metabolites.get_by_id('3h2mpentACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mpent2eACP_c:1
})

In [215]:
#last step: convert to 2-methyl-pentanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 2-methyl-pentanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mpentACP_c', name = '2-methyl-pentanoyl-ACP(iso-C65:0ACP)', compartment = 'c', charge = -1, formula = 'C17H31O8N2PRS' ))

In [216]:
model.metabolites.get_by_id('2mpentACP_c').annotation['sbo'] = 'SBO:0000247'

In [217]:
#add reaction with NADH: EMAR60x
model.add_reaction(Reaction(id='EMAR60x', name = '2-methyl-pentanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [218]:
model.reactions.EMAR60x.annotation['sbo'] = 'SBO:0000176'

In [219]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR60x.gene_reaction_rule = 'RTMO12345'

In [220]:
model.reactions.EMAR60x.add_metabolites({
    model.metabolites.t2mpent2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mpentACP_c'):1,
    model.metabolites.h_c:-1
})

In [221]:
#add reaction with NADH: EMAR60Y
model.add_reaction(Reaction(id='EMAR60y', name = '2-methyl-pentanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [222]:
model.reactions.EMAR60y.annotation['sbo'] = 'SBO:0000176'

In [223]:
#assume GPR is the same FAsynthase: RTMO12345
model.reactions.EMAR60y.gene_reaction_rule = 'RTMO12345'

In [224]:
model.reactions.EMAR60y.add_metabolites({
    model.metabolites.t2mpent2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mpentACP_c'):1,
    model.metabolites.h_c:-1
})

In [225]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

finished to iso-C:60. now onto iso-c8:0.

In [226]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [227]:
#add condenstation of 2mpent-ACP with malonyl ACP to 3-oxo-2-methyl-heptanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mheptACP_c', name = '3-oxo-2-methylheptanoyl-ACP', compartment = 'c', charge =-1, formula = 'C19H33O9N2PRS'))

In [228]:
model.metabolites.get_by_id('3o2mheptACP_c').annotation['sbo'] = 'SBO:0000176'

In [229]:
#add reaction
model.add_reaction(Reaction(id='3OMAS80', name = '3-oxo-2-methylheptanoyl-[acyl-carrier-protein] synthase (iso-C8:0)'))

Ignoring reaction '3OMAS80' since it already exists.


In [230]:
model.reactions.get_by_id('3OMAS80').annotation['sbo'] = 'SBO:0000176'

In [231]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OMAS80').gene_reaction_rule = 'RTMO12345'

In [232]:
model.reactions.get_by_id('3OMAS80').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mpentACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mheptACP_c'):1,
    model.metabolites.h_c:-1
})

In [233]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methylheptanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mheptACP_c', name = '3-Hydroxy-2-methyl-heptanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C19H35O9N2PRS'))

In [234]:
model.metabolites.get_by_id('3h2mheptACP_c').annotation['sbo'] = 'SBO:0000247'

In [235]:
#add reaction
model.add_reaction(Reaction(id='3OMAR80', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (iso-C8:0)'))

In [236]:
model.reactions.get_by_id('3OMAR80').annotation['sbo'] = 'SBO:0000176'

In [237]:
#assume GPR is the same FAget_by_id('3OAS70')synthase: RTMO12345
model.reactions.get_by_id('3OMAR80').gene_reaction_rule = 'RTMO12345'

In [238]:
model.reactions.get_by_id('3OMAR80').add_metabolites({
    model.metabolites.get_by_id('3o2mheptACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mheptACP_c'):1,
    model.metabolites.h_c:-1
})

In [239]:
#next add the tpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mhept2eACP_c', name = 'Trans-2-methyl-Hept-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C19H33O8N2PRS'))

In [240]:
model.metabolites.get_by_id('t2mhept2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [241]:
#add reaction
model.add_reaction(Reaction(id='3HMAD80', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (iso-C8:0)'))

In [242]:
model.reactions.get_by_id('3HMAD80').annotation['sbo'] = 'SBO:0000176'

In [243]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD80').gene_reaction_rule = 'RTMO12345'

In [244]:
model.reactions.get_by_id('3HMAD80').add_metabolites({
    model.metabolites.get_by_id('3h2mheptACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mhept2eACP_c:1
})

In [245]:
#last step: convert to 2-methyl-Heptanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 2-methyl-heptanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mheptACP_c', name = '2-methyl-heptanoyl-ACP(iso-C8:0ACP)', compartment = 'c', charge = -1, formula = 'C19H35O8N2PRS' ))

In [246]:
model.metabolites.get_by_id('2mheptACP_c').annotation['sbo'] = 'SBO:0000247'

In [247]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR80x', name = '2-methyl-heptanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [248]:
model.reactions.EMAR80x.annotation['sbo'] = 'SBO:0000176'

In [249]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR80x.gene_reaction_rule = 'RTMO12345'

In [250]:
model.reactions.EMAR80x.add_metabolites({
    model.metabolites.t2mhept2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mheptACP_c'):1,
    model.metabolites.h_c:-1
})

In [251]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR80y', name = '2-methyl-heptanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [252]:
model.reactions.EMAR80y.annotation['sbo'] = 'SBO:0000176'

In [253]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR80y.gene_reaction_rule = 'RTMO12345'

In [254]:
model.reactions.EMAR80y.add_metabolites({
    model.metabolites.t2mhept2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mheptACP_c'):1,
    model.metabolites.h_c:-1
})

In [255]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished until iso-C8:0. Now onto iso-C10:0.

In [256]:
#add condenstation of 2mhept-ACP with malonyl ACP to 3-oxo-2-methyl-nonanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mnonACP_c', name = '3-oxo-2-methyl-nonanoyl-ACP', compartment = 'c', charge =-1, formula = 'C21H37O9N2PRS'))

In [257]:
model.metabolites.get_by_id('3o2mnonACP_c').annotation['sbo'] = 'SBO:0000176'

In [258]:
#add reaction
model.add_reaction(Reaction(id='3OMAS100', name = '3-oxo-2-methyl-nonanoyl-[acyl-carrier-protein] synthase (iso-C10:0)'))

In [259]:
model.reactions.get_by_id('3OMAS100').annotation['sbo'] = 'SBO:0000176'

In [260]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS100').gene_reaction_rule = 'RTMO12345'

In [261]:
model.reactions.get_by_id('3OMAS100').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mheptACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mnonACP_c'):1,
    model.metabolites.h_c:-1
})

In [262]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methyl-nonanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mnonACP_c', name = '3-Hydroxy-2-methyl-nonanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C21H39O9N2PRS'))

In [263]:
model.metabolites.get_by_id('3h2mnonACP_c').annotation['sbo'] = 'SBO:0000247'

In [264]:
#add reaction
model.add_reaction(Reaction(id='3OMAR100', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (iso-C10:0)'))

In [265]:
model.reactions.get_by_id('3OMAR100').annotation['sbo'] = 'SBO:0000176'

In [266]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OMAR100').gene_reaction_rule = 'RTMO12345'

In [267]:
model.reactions.get_by_id('3OMAR100').add_metabolites({
    model.metabolites.get_by_id('3o2mnonACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mnonACP_c'):1,
    model.metabolites.h_c:-1
})

In [268]:
#next add the t2mpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mnon2eACP_c', name = 'Trans-2-methyl-Non-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C21H37O8N2PRS'))

In [269]:
model.metabolites.get_by_id('t2mnon2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [270]:
#add reaction
model.add_reaction(Reaction(id='3HMAD100', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (iso-C10:0)'))

In [271]:
model.reactions.get_by_id('3HMAD100').annotation['sbo'] = 'SBO:0000176'

In [272]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD100').gene_reaction_rule = 'RTMO12345'

In [273]:
model.reactions.get_by_id('3HMAD100').add_metabolites({
    model.metabolites.get_by_id('3h2mnonACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mnon2eACP_c:1
})

In [274]:
#add both with NADH and NADPH.
#first add the 2-methyl-nonanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mnonACP_c', name = '2-methyl-nonanoyl-ACP(iso-C10:0ACP)', compartment = 'c', charge = -1, formula = 'C21H39O8N2PRS' ))

In [275]:
model.metabolites.get_by_id('2mnonACP_c').annotation['sbo'] = 'SBO:0000247'

In [276]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR100x', name = '2-methylnonanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [277]:
model.reactions.EMAR100x.annotation['sbo'] = 'SBO:0000176'

In [278]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR100x.gene_reaction_rule = 'RTMO12345'

In [279]:
model.reactions.EMAR100x.add_metabolites({
    model.metabolites.t2mnon2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mnonACP_c'):1,
    model.metabolites.h_c:-1
})

In [280]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR100y', name = '2-methyl-nonanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [281]:
model.reactions.EMAR100y.annotation['sbo'] = 'SBO:0000176'

In [282]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR100y.gene_reaction_rule = 'RTMO12345'

In [283]:
model.reactions.EMAR100y.add_metabolites({
    model.metabolites.t2mnon2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mnonACP_c'):1,
    model.metabolites.h_c:-1
})

In [284]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished until iso-C10:0, now move onto iso -C12:0. 

In [285]:
#add condenstation of 2-methyl-non-ACP with malonyl ACP to 3-oxo2-methyl-undecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mudecACP_c', name = '3-oxo-2-methyl-undecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C23H41O9N2PRS'))

In [286]:
model.metabolites.get_by_id('3o2mudecACP_c').annotation['sbo'] = 'SBO:0000176'

In [287]:
#add reaction
model.add_reaction(Reaction(id='3OMAS120', name = '3-oxo-2-methyl-undecanoyl-[acyl-carrier-protein] synthase (iso-C12:0)'))

In [288]:
model.reactions.get_by_id('3OMAS120').annotation['sbo'] = 'SBO:0000176'

In [289]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS120').gene_reaction_rule = 'RTMO12345'

In [290]:
model.reactions.get_by_id('3OMAS120').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mnonACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mudecACP_c'):1,
    model.metabolites.h_c:-1
})

In [317]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methyl-undecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mudecACP_c', name = '3-Hydroxy-2-methyl-undecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C23H43O9N2PRS'))

In [318]:
model.metabolites.get_by_id('3h2mudecACP_c').annotation['sbo'] = 'SBO:0000247'

In [319]:
#add reaction
model.add_reaction(Reaction(id='3OMAR120', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (iso-C12:0)'))

In [320]:
model.reactions.get_by_id('3OMAR120').annotation['sbo'] = 'SBO:0000176'

In [321]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OMAR120').gene_reaction_rule = 'RTMO12345'

In [322]:
model.reactions.get_by_id('3OMAR120').add_metabolites({
    model.metabolites.get_by_id('3o2mudecACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mudecACP_c'):1,
    model.metabolites.h_c:-1
})

In [323]:
#next add the t2mudec2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mudec2eACP_c', name = 'Trans-2-methyl-Undec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C23H41O8N2PRS'))

In [324]:
model.metabolites.get_by_id('t2mudec2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [325]:
#add reaction
model.add_reaction(Reaction(id='3HMAD120', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (iso-C12:0)'))

In [326]:
model.reactions.get_by_id('3HMAD120').annotation['sbo'] = 'SBO:0000176'

In [301]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD120').gene_reaction_rule = 'RTMO12345'

In [327]:
model.reactions.get_by_id('3HMAD120').add_metabolites({
    model.metabolites.get_by_id('3h2mudecACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mudec2eACP_c:1
})

In [328]:
#add both with NADH and NADPH.
#first add the 2-methylundecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mudecACP_c', name = '2-Methyl-Undecanoyl-ACP(iso-C12:0ACP)', compartment = 'c', charge = -1, formula = 'C23H43O8N2PRS' ))

In [329]:
model.metabolites.get_by_id('2mudecACP_c').annotation['sbo'] = 'SBO:0000247'

In [330]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR120x', name = '2-methyl-Undecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [331]:
model.reactions.EMAR120x.annotation['sbo'] = 'SBO:0000176'

In [332]:
#assume GPR is the same FAsynthase: RTMO12345
model.reactions.EMAR120x.gene_reaction_rule = 'RTMO12345'

In [333]:
model.reactions.EMAR120x.add_metabolites({
    model.metabolites.t2mudec2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mudecACP_c'):1,
    model.metabolites.h_c:-1
})

In [334]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR120y', name = '2-Methyl-Undecanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [335]:
model.reactions.EMAR120y.annotation['sbo'] = 'SBO:0000176'

In [336]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR120y.gene_reaction_rule = 'RTMO12345'

In [338]:
model.reactions.EMAR120y.add_metabolites({
    model.metabolites.t2mudec2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mudecACP_c'):1,
    model.metabolites.h_c:-1
})

In [313]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished to isoC12:0, now onto isoC14:0. 

In [339]:
#add condenstation of 2mnon-ACP with malonyl ACP to 3-oxo-2-methyltridecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mtridecACP_c', name = '3-oxo-2-methyltridecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C25H45O9N2PRS'))

In [340]:
model.metabolites.get_by_id('3o2mtridecACP_c').annotation['sbo'] = 'SBO:0000176'

In [341]:
#add reaction
model.add_reaction(Reaction(id='3OMAS140', name = '3-oxo-2-methyl-tridecanoyl-[acyl-carrier-protein] synthase (iso-C15:0)'))

In [342]:
model.reactions.get_by_id('3OMAS140').annotation['sbo'] = 'SBO:0000176'

In [343]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS140').gene_reaction_rule = 'RTMO12345'

In [345]:
model.reactions.get_by_id('3OMAS140').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mudecACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mtridecACP_c'):1,
    model.metabolites.h_c:-1
})

In [346]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methyl-tridecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mtridecACP_c', name = '3-Hydroxy-2-methyl-tridecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C25H47O9N2PRS'))

In [347]:
model.metabolites.get_by_id('3h2mtridecACP_c').annotation['sbo'] = 'SBO:0000247'

In [348]:
#add reaction
model.add_reaction(Reaction(id='3OMAR140', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (iso-C14:0)'))

In [349]:
model.reactions.get_by_id('3OMAR140').annotation['sbo'] = 'SBO:0000176'

In [350]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OMAR140').gene_reaction_rule = 'RTMO12345'

In [351]:
model.reactions.get_by_id('3OMAR140').add_metabolites({
    model.metabolites.get_by_id('3o2mtridecACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mtridecACP_c'):1,
    model.metabolites.h_c:-1
})

In [353]:
#next add the 2mtpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mtridec2eACP_c', name = 'Trans-2-methyl-Tridec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C25H45O8N2PRS'))

In [354]:
model.metabolites.get_by_id('t2mtridec2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [355]:
#add reaction
model.add_reaction(Reaction(id='3HMAD140', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (iso-C14:0)'))

In [356]:
model.reactions.get_by_id('3HMAD140').annotation['sbo'] = 'SBO:0000176'

In [357]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD140').gene_reaction_rule = 'RTMO12345'

In [358]:
model.reactions.get_by_id('3HMAD140').add_metabolites({
    model.metabolites.get_by_id('3h2mtridecACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mtridec2eACP_c:1
})

In [359]:
#add the last reaction both with NADH and NADPH.
#first add the 2-methyl-tridecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mtridecACP_c', name = '2-methyl-Tridecanoyl-ACP(iso-C14:0ACP)', compartment = 'c', charge = -1, formula = 'C25H47O8N2PRS' ))

In [360]:
model.metabolites.get_by_id('2mtridecACP_c').annotation['sbo'] = 'SBO:0000247'

In [361]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR140x', name = '2-methyl-Tridecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [362]:
model.reactions.EMAR140x.annotation['sbo'] = 'SBO:0000176'

In [363]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR140x.gene_reaction_rule = 'RTMO12345'

In [364]:
model.reactions.EMAR140x.add_metabolites({
    model.metabolites.t2mtridec2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mtridecACP_c'):1,
    model.metabolites.h_c:-1
})

In [365]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR140y', name = '2-Methyl-Tridecanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [366]:
model.reactions.EMAR140y.annotation['sbo'] = 'SBO:0000176'

In [367]:
#assume GPR is the same FAsynthase: RTMO12345
model.reactions.EMAR140y.gene_reaction_rule = 'RTMO12345'

In [369]:
model.reactions.EMAR140y.add_metabolites({
    model.metabolites.t2mtridec2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mtridecACP_c'):1,
    model.metabolites.h_c:-1
})

In [93]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

finished up to isoC14:0, now move onto the final iso C16:0 fatty acid.

In [370]:
#add condenstation of 2mtridec-ACP with malonyl ACP to 3-oxo-2-methylpentadecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mpdecACP_c', name = '3-oxo-2-methyl-pentadecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C27H49O9N2PRS'))

In [371]:
model.metabolites.get_by_id('3o2mpdecACP_c').annotation['sbo'] = 'SBO:0000176'

In [372]:
#add reaction
model.add_reaction(Reaction(id='3OMAS160', name = '3-oxo-2-methyl-pentadecanoyl-[acyl-carrier-protein] synthase (iso-C16:0)'))

In [373]:
model.reactions.get_by_id('3OMAS160').annotation['sbo'] = 'SBO:0000176'

In [374]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS160').gene_reaction_rule = 'RTMO12345'

In [376]:
model.reactions.get_by_id('3OMAS160').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mtridecACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mpdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [377]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methyl-pentadecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mpdecACP_c', name = '3-Hydroxy-2-methyl-pentadecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C27H51O9N2PRS'))

In [378]:
model.metabolites.get_by_id('3h2mpdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [379]:
#add reaction
model.add_reaction(Reaction(id='3OMAR160', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (iso-C16:0)'))

In [380]:
model.reactions.get_by_id('3OMAR160').annotation['sbo'] = 'SBO:0000176'

In [381]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OMAR160').gene_reaction_rule = 'RTMO12345'

In [382]:
model.reactions.get_by_id('3OMAR160').add_metabolites({
    model.metabolites.get_by_id('3o2mpdecACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mpdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [383]:
#next add the next metabolite
model.add_metabolites(Metabolite(id='t2mpdec2eACP_c', name = 'Trans-2-methyl-Pentadec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C27H49O8N2PRS'))

In [384]:
model.metabolites.get_by_id('t2mpdec2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [385]:
#add reaction
model.add_reaction(Reaction(id='3HMAD160', name = '3-hydroxy-methyl-acyl-[acyl-carrier-protein] dehydratase (iso-C16:0)'))

In [386]:
model.reactions.get_by_id('3HMAD160').annotation['sbo'] = 'SBO:0000176'

In [387]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD160').gene_reaction_rule = 'RTMO12345'

In [388]:
model.reactions.get_by_id('3HMAD160').add_metabolites({
    model.metabolites.get_by_id('3h2mpdecACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mpdec2eACP_c:1
})

In [389]:
#add the last reaction both with NADH and NADPH.
#first add the 2-methyl-pentadecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mpdecACP_c', name = '2-Methyl-Pentadecanoyl-ACP(iso-C16:0ACP)', compartment = 'c', charge = -1, formula = 'C27H51O8N2PRS' ))

In [390]:
model.metabolites.get_by_id('2mpdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [391]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR160x', name = '2-methyl-Pentadecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [392]:
model.reactions.EMAR160x.annotation['sbo'] = 'SBO:0000176'

In [393]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR160x.gene_reaction_rule = 'RTMO12345'

In [394]:
model.reactions.EMAR160x.add_metabolites({
    model.metabolites.t2mpdec2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mpdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [395]:
#add reaction with NADH
model.add_reaction(Reaction(id='EMAR160y', name = '2-Methyl-Pentadecanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [396]:
model.reactions.EMAR160y.annotation['sbo'] = 'SBO:0000176'

In [397]:
#assume GPR is the same FAsynthase: RTMO12345
model.reactions.EMAR160y.gene_reaction_rule = 'RTMO12345'

In [398]:
model.reactions.EMAR160y.add_metabolites({
    model.metabolites.t2mpdec2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mpdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [122]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

finished:
    - isoC15:0: met 3mtdecACP_c
    - isoC17:0: met 3mhdecACP_c
    - iso C14:0: met 2mtridecACP_c
    - iso C16:0: met 2mpdecACP_c

In [399]:
#check no change in simulation
model.optimize()

,fluxes,reduced_costs
IDPh,1.770060e+00,0.000000e+00
CAT,1.979106e-02,0.000000e+00
PDHam1hi,0.000000e+00,2.775558e-17
HYDA,0.000000e+00,-0.000000e+00
MALHYDRO,8.881784e-16,0.000000e+00
...,...,...
3OMAS160,0.000000e+00,-5.700687e-02
3OMAR160,0.000000e+00,-5.700687e-03
3HMAD160,0.000000e+00,0.000000e+00
EMAR160x,0.000000e+00,-1.368165e-02
